## 勾配を求める

w を重み、b をバイアス、E を誤差とすると、勾配降下法での更新式はこんな感じ。

$$
  w \leftarrow w - \eta \frac{\partial E}{\partial w} \\
  b \leftarrow b - \eta \frac{\partial E}{\partial b}
$$

三層 NN を想定して、各パラメータを以下の様に定義してみる

|層|添字|ニューロン数|
|----|----|----|
|入力|i|l|
|中間|j|m|
|出力|k|n|

入力層は勾配もクソもないので、中間層と出力層を考えてみる。

### 出力層の勾配

$y_j$ で受け取って、$w_{jk}$ を掛けて、$b_k$ 足した結果 $u_k$ を活性関数に食わせたのが $y_k$ と、文字にしてみると却って分かりにくいな。

$$
u_k = y_j w_{jk} + b_k \\
y_k = f(u_k)
$$

重みを1パラメータづつ勾配を求めるとするなら要するに偏微分なので

$$
  \partial w_{jk} = \frac{\partial E}{\partial w_{jk}}  \\
  = \frac{\partial E}{\partial u_k} \frac{\partial u_k}{\partial w_{jk}}  (※)
$$

※連鎖律を適用。連鎖律については [ここ](https://white-azalea.hatenablog.jp/entry/2021/02/11/171005) 参照

$ \frac{\partial u_k}{\partial w_{jk}} $ は $ u_k $ を展開すると

$$
\frac{\partial u_k}{\partial w_{jk}} = \frac{\partial (\sum^m_{q=1} y_q w_{qk} + b_k) }{\partial w_{jk}} \\
= \frac{\partial}{\partial w_{jk}} (y_1 w_{1k} + y_2 w_{2k} ... y_j w_{jk} ... + y_m w_mk + b_k)
$$

偏微分なので、$ w_{jk} $ のかかってない項はすべて 0 なので、

$$
\frac{\partial u_k}{\partial w_{jk}} = \frac{\partial}{\partial w_{jk}} (y_1 w_{1k} + y_2 w_{2k} ... y_j w_{jk} ... + y_m w_mk + b_k)
= y_j
$$

次に $ \frac{\partial E}{\partial u_k} $ に焦点を当てて、こちらも連鎖律展開して、$ \delta_k $ としていれば

$$
\delta_k = \frac{\partial E}{\partial u_k} = \frac{\partial E}{\partial y_k} \frac{\partial y_k}{\partial u_k}
$$

すると

$$
\partial w_{jk} = y_j \delta_k
$$

という形に持っていける。

$ \partial b_k $ も同様の手順で考えると

$$
\partial b_k = \frac{\partial E}{\partial b_k} = \frac{\partial E}{\partial u_k} \frac{\partial u_k}{\partial b_k}
$$

$u_k$ を展開すると、最終的に 1 になるので、最終形態は $ \partial b_k = \delta_k $

### 中間層出力の勾配

出力層では中間層の演算のために事前に $ \frac{\partial E}{\partial y_j} $ （中間層出力勾配）を計算しておく。  
扱いが面倒なので、これを $ \partial y_j $ としておいておく。

こいつにまず連鎖律を適用して

$$
\partial y_j = \frac{\partial E}{\partial y_j} = \sum^n_{r=i} \frac{\partial E}{\partial u_r} \frac{\partial u_r}{\partial y_j}
$$

このうち、

$$
\frac{\partial u_r}{\partial y_j} = \frac{\partial (\sum^m_{q=1}) y_q w_{qr} + b_r}{\partial y_j} \\
= \frac{\partial}{\partial y_j} (y_1 w_{1r} + y_2 w_{2r} + ... y_j w_{jr} + ... + y_m w_{wr} + b_r)  \\
= w_{jr}  (y_j の偏微分なので、y_j を含まない項はすべて 0 になる)
$$

ここに簡略化のために $ \delta_r = \frac{\partial E}{\partial u_r} $ とすれば

$$
\partial y_j = \sum^n_{r=1} \delta_r w_{jr}
$$



### 中間層の勾配

出力層同様に、重み勾配 $ \partial w_{ij} $ は

$$
\partial w_{ij} = \frac{\partial E}{\partial w_{ij}} = \frac{\partial E}{\partial u_j} \frac{\partial u_j}{\partial w_{ij}}
$$

このうち $\frac{\partial u_j}{\partial e_{ij}}$ は

$$
\frac{\partial u_j}{\partial w_{ij}} = \frac{\partial (\sum^l_{p=1} y_p w_{pj} + b_j)}{\partial w_{ij}} \\
  = y_i (展開して偏微分するとこうなる)
$$

もう一つ $\frac{\partial E}{\partial u_j}$ 部分は連鎖律使って

$$
\frac{\partial E}{\partial u_j} = \frac{\partial E}{\partial y_j} \frac{\partial y_j}{\partial u_j}
$$

このうち $ \frac{\partial y_j}{\partial u_j} $ は活性関数の微分。  
$ \frac{\partial E}{\partial y_j} $ は中間層出力の勾配(= $ \partial y_j $ )。

$$
\delta_j = \frac{\partial E}{\partial u_j} = \partial y_j \frac{\partial y_j}{\partial u_j}
$$

と、なるほど。中間層出力の勾配を受け取って、中間層が計算できる → 下層からの結果を受け取る → 逆伝播の原理 というわけですね

$\frac{\partial u_j}{\partial w_{ij}} = y_i$ を使うと $ \partial w_{ij} = y_i \delta_j $

バイアス勾配 $ \partial b_j $ も

$$
\partial b_j = \frac{\partial E}{\partial b_j} = \frac{\partial E}{\partial u_j} \frac{\partial u_j}{\partial b_j}
$$

このうち

$$
\frac{\partial u_j}{\partial b_j} = \frac{\partial (\sum^l_{p=1} y_p w_{pj} + b_j )}{\partial b_j} \\
  = 1 (※)
$$

※ 偏微分なので、$ b_j $ 以外の全ての項は 0 になり、残った $ b_j $ も微分されるため。

そのため

$$
$ \partial b_j = \delta_j $
$$

こんなノリで上の層に伝播を続けていく。

## 勾配計算式

$w$ を重み、$b$ をバイアス、$E$ を誤差(損失関数出力)とするとこんな形状で定式化できる。  
ここに至る計算については、[はじめてのディープラーニング](https://amzn.to/2OsglAs) が丁寧に教えてくれた。  
（このブログ書いてる本人の数学レベルは、「偏微分？あーそういやあったね」程度のクソザコレベルが時間を掛ければ付いていける程度に順を追って書いてくれてる）

数式を飲み込むのにはそれなりに時間を要したけど…。

* 出力層

$$
\delta_k = \frac{\partial E}{\partial u_k} = \frac{\partial E}{\partial y_k} \frac{\partial y_k}{\partial u_k} \\
\partial w_{jk} = \frac{\partial E}{\partial w_{jk}} = y_j \delta_k (重み勾配) \\
\partial b_k = \frac{\partial E}{\partial b_k} = \delta_k (バイアス勾配) \\
\partial y_j = \frac{\partial E}{\partial y_j} = \sum^n_{r=1} \delta_r w_{jr} (一つ上の層の出力勾配)
$$

* 中間層

$$
\delta_j = \frac{\partial E}{\partial u_j} = \partial y_j \frac{\partial y_k}{\partial u_j} \\
\partial w_{ij} = \frac{\partial E}{\partial w_{ij}} = y_i \delta_j (重み勾配) \\
\partial b_j = \frac{\partial E}{\partial b_j} = \delta_j (バイアス勾配) \\
\partial y_i = \frac{\partial E}{\partial y_i} = \sum^m_{q=1} \delta_q w_{iq} (一つ上の層の出力勾配)
$$

となる。

### 回帰型ニューラルネットの勾配

* 損失関数 = 二乗和誤差
* 中間層 = シグモイド関数
* 出力層 = 恒等関数

$$
\delta_k = \frac{\partial E}{\partial u_k} = \frac{\partial E}{\partial y_k} \frac{\partial y_k}{\partial u_k}
$$

から考える。損失関数の二乗和誤差を $y_k$ で偏微分する ($y$ が結果値で $t$ が理想値)

$$
\frac{\partial E}{\partial y_k} = \frac{\partial}{\partial y_k} (\frac{1}{2} \sum_k(y_k - t_k)^2) \\
   =  \frac{\partial}{\partial y_k} (\frac{1}{2}(y_0 - t_0)^2 + \frac{1}{2}(y_1 - t_1)^2 + ... \frac{1}{2}(y_n - t_n)^2) \\
   = y_k - t_k (※)
$$

※ 偏微分につき、$ y_k $ を含まない項はすべて 0 となるため。

活性関数が恒等関数（入力 = 出力）の関数なので、

$$
\frac{\partial y_k}{\partial u_k} = 1
$$

すると勾配の式は

$$
\delta_k = y_k - t_k \\
\partial w_{jk} = y_j \delta_k (重み勾配) \\
\partial b_k = \delta_k (バイアス勾配) \\
\partial y_j = \sum^n_{r=1} \delta_r w_{jr} (一つ上の層の出力勾配)
$$

次に中間層を考える。この場合も $\delta_j$ を求める。  
シグモイド関数 $f$ の微分は $ f'(x) = (1-f(x))f(x) $ なので

$$
\frac{\partial y_k}{\partial u_j} = (1 - y_j)y_j \\
\delta_j = \partial y_j \frac{\partial y_k}{\partial u_j} = \partial y_j (1 - y_j)y_j
$$

なので各勾配は

$$
\delta_j = \partial y_j (1 - y_j)y_j \\
\partial w_{ij} = y_i \delta_j (重み勾配) \\
\partial b_j = \delta_j (バイアス勾配) \\
\partial y_i = \sum^m_{q=1} \delta_q w_{iq} (一つ上の層の出力勾配)
$$

### 分類型ニューラルネットの勾配

* 損失関数 = 交差エントロピー誤差
* 中間層 = シグモイド関数
* 出力層 = ソフトマックス関数

#### 交差エントロピー誤差

[こちらの記事](https://qiita.com/kenta1984/items/59a9ef1788e6934fd962) が丁寧に書いてくれています。

数式的にはこんな関数

$$
E(t,y) = - \sum_x t_x log(y_x)
$$

理想の出力 $ t $ は、スイッチ出力なので `[1, 0, 0, 0]` の様な一つだけ 1 となってるので、仮に入力値を `[0.5, 0.8, 0.2, 0.4]` だとすると

$$
E(t,y) = - (1 log(0.5) + 0 log(0.8) + 0 log(0.2) + 0 log(0.4) \\
  = -log(0.5) \\
  = -(-0.693...) \\
  = 0.693
$$

こんな感じで出てくる。

#### ソフトマックス関数

[過去記事](https://white-azalea.hatenablog.jp/entry/2021/02/17/205517) 参照(グラフ付き)。

$$
y = \frac{exp(x)}{\sum^n_{k=1} exp(k)}
$$


こんな感じの関数を組み合わせすると

$$
E = - \sum_k t_k log( \frac{exp(u_k)}{ \sum_k exp(u_k) } ) \\
  = - \sum_k ( t_k log(exp(u_k)) - t_k log(\sum_k exp(u_k)) ) ... log \frac{p}{q} = log(p) - log(q) による \\
  = なんやかんやあって \\
  = - \sum_k t_k u_k + log(\sum_k exp(u_k))
$$

これを更新式に突っ込むと

$$
\delta_k = \frac{\partial E}{\partial y_k} \frac{\partial y_k}{\partial u_k} \\
= \frac{\partial}{\partial u_k} (- \sum_k t_k u_k + log(\sum_k exp(u_k))) \\
= -t_k + \frac{exp(u_k)}{\sum_k exp(u_k)} ... 見覚えのある形出現\\
= -t_k + y_k
$$

後は↑の公式に当てはめればいい。

中間層は、シグモイド関数なので、回帰の場合と同一